In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm



In [ ]:

## SPLITTING THE DATA

import os
import shutil
import random
from pathlib import Path

SOURCE_DIR = "/Users/niakumar/Documents/Endometrial_US_DATA/TRAIN/Benign"
TARGET_DIR = "/Users/niakumar/Documents/Ultrasound_Split"
CLASS_NAME = "Benign"
VAL_SPLIT = 0.2
SEED = 42

# destination paths

train_dir = Path(TARGET_DIR) / "train" / CLASS_NAME
val_dir = Path(TARGET_DIR) / "validate" / CLASS_NAME
train_dir.mkdir(parents =True, exist_ok=True)
val_dir.mkdir(parents =True, exist_ok=True)

# split files

random.seed(SEED)
all_files = [f for f in os.listdir(SOURCE_DIR) if f.lower().endswith(('.bmp', '.png', '.jpg', '.jpeg'))]
random.shuffle(all_files)
val_count = int(len(all_files) * VAL_SPLIT)
val_files = set(all_files[:val_count])

for fname in all_files:
    src_path = os.path.join(SOURCE_DIR, fname)
    dst_path = os.path.join(val_dir if fname in val_files else train_dir, fname)
    shutil.copy2(src_path, dst_path)

print(f"Total images: {len(all_files)}")
print(f"Copied to:")
print(f" ->Val: {len(val_files)} -> {val_dir}")
print(f" ->Train: {len(all_files) - len(val_files)} -> {train_dir}")






In [ ]:
# Data transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load data
train_data = datasets.ImageFolder(Path(TARGET_DIR) / "train", transform=transform)
val_data = datasets.ImageFolder(Path(TARGET_DIR) / "validate", transform=transform)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)


In [ ]:
# Model
model = models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## ADAMW OPTIMIZER WITH COSINE ANNEALING SCHEDULER ##n

from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

from torch.optim.lr_scheduler import CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.4, 0.6]).to(device))


In [ ]:
# Training loop
for epoch in range(30):  # or more
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")


In [ ]:
# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {correct / total:.2%}")

# Save the trained model
torch.save(model.state_dict(), 'resnet_ultrasound.pt')
print("✅ Model saved as resnet_ultrasound.pt")

In [ ]:
import torch
import numpy as np
import umap
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn

# --- UMAP Plotting Function ---
def plot_umap(embeddings, labels, plot_3d=True, title="UMAP Projection"):
    reducer = umap.UMAP(n_components=3 if plot_3d else 2, random_state=42)
    embedding_umap = reducer.fit_transform(embeddings)

    fig = plt.figure(figsize=(10, 7))
    cmap_choice = 'tab10'  # Better for class contrast
    if plot_3d:
        from mpl_toolkits.mplot3d import Axes3D
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(
            embedding_umap[:, 0], embedding_umap[:, 1], embedding_umap[:, 2],
            c=labels, cmap=cmap_choice, s=40, alpha=0.9
        )
    else:
        scatter = plt.scatter(
            embedding_umap[:, 0], embedding_umap[:, 1],
            c=labels, cmap=cmap_choice, s=40, alpha=0.9
        )

    plt.title(title)
    plt.colorbar(scatter)
    plt.show()

# --- Use the pretrained model up to the penultimate layer ---
embedding_model = nn.Sequential(*list(model.children())[:-1])  # Remove final FC layer
embedding_model.eval().to(device)

# --- Extract embeddings from validation data ---
embeddings = []
labels = []

with torch.no_grad():
    for images, lbls in val_loader:
        images = images.to(device)

        # Extract features from penultimate layer
        feats = embedding_model(images)           # shape: (B, 512, 1, 1)
        feats = feats.view(feats.size(0), -1)     # flatten to (B, 512)

        embeddings.append(feats.cpu().numpy())
        labels.extend(lbls.numpy())

embeddings = np.vstack(embeddings)
labels = np.array(labels)

# --- Plot UMAP ---
plot_umap(embeddings, labels, plot_3d=True, title="3D UMAP of Validation Features")


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import umap
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import pairwise_distances
import shutil

# === Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Adjust these paths:
CLEAN_DATA_DIR = "/Users/niakumar/Documents/Ultrasound_Split/train"  # parent of class folders
NEW_DATA_DIR = "/Users/niakumar/Documents/New_Mask_Mult_Crop_BENIGN"

# === Image transforms (same for clean and new) ===
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# === Custom Dataset class to return file paths ===
from torchvision.datasets import ImageFolder
class ImageFolderWithPaths(ImageFolder):
    def __getitem__(self, index):
        img, label = super().__getitem__(index)
        path = self.imgs[index][0]
        return img, label, path

# === Load clean dataset (with filepaths if needed) ===
clean_dataset = ImageFolderWithPaths(CLEAN_DATA_DIR, transform=transform)
clean_loader = DataLoader(clean_dataset, batch_size=32, shuffle=False)

# === Load new dataset ===
from torch.utils.data import Dataset
from PIL import Image
import glob

class FlatImageDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.image_paths = glob.glob(os.path.join(folder, "*.bmp"))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)
        return img, 0, img_path  # dummy label '0' to match clean dataset output format
    
new_dataset = FlatImageDataset(NEW_DATA_DIR, transform=transform)
new_loader = DataLoader(new_dataset, batch_size=32, shuffle=False)

# === Load your trained ResNet18 model ===
# Replace this with your actual model loading code if needed
from torchvision.models import resnet50
model = resnet50(pretrained=False)
model.conv1 = nn.Conv2d(
        1,
        64,
        kernel_size=7,
        stride=2,
        padding=3,
        bias=False
    )

model.fc = nn.Linear(in_features=2048, out_features=2)
# Load your trained weights here, e.g.:
# model.load_state_dict(torch.load("your_model_path.pth"))
model.load_state_dict(torch.load("/Users/niakumar/Documents/Ultrasound_Split/resnet_ultrasound.pt", map_location=device))
model = model.to(device)
model.eval()

# === Create embedding extractor (all layers except final fc) ===
embedding_model = nn.Sequential(*list(model.children())[:-1])
embedding_model = embedding_model.to(device)
embedding_model.eval()

def extract_embeddings(dataloader):
    embeddings = []
    file_paths = []
    with torch.no_grad():
        for images, labels, paths in dataloader:
            images = images.to(device)
            feats = embedding_model(images)
            feats = feats.view(feats.size(0), -1)
            embeddings.append(feats.cpu().numpy())
            file_paths.extend(paths)
    embeddings = np.vstack(embeddings)
    return embeddings, file_paths

# === Extract embeddings ===
print("Extracting embeddings for clean dataset...")
clean_embeddings, clean_paths = extract_embeddings(clean_loader)

print("Extracting embeddings for new dataset...")
new_embeddings, new_paths = extract_embeddings(new_loader)

# === Combine for UMAP ===
combined_embeddings = np.vstack([clean_embeddings, new_embeddings])
combined_labels = [0]*len(clean_embeddings) + [1]*len(new_embeddings)  # 0=clean,1=new

def plot_umap(embeddings, labels, plot_3d=False, title="UMAP Projection"):
    reducer = umap.UMAP(n_components=3 if plot_3d else 2, random_state=42)
    embedding_umap = reducer.fit_transform(embeddings)

    plt.figure(figsize=(10,7))
    cmap_choice = 'tab10'
    if plot_3d and embeddings.shape[1] >= 3:
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure(figsize=(10,7))
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(
            embedding_umap[:,0], embedding_umap[:,1], embedding_umap[:,2],
            c=labels, cmap=cmap_choice, s=40, alpha=0.9)
    else:
        scatter = plt.scatter(
            embedding_umap[:,0], embedding_umap[:,1],
            c=labels, cmap=cmap_choice, s=40, alpha=0.9)

    plt.title(title)
    plt.colorbar(scatter, ticks=[0,1], label='Dataset (0=Clean, 1=New)')
    plt.show()

print("Plotting UMAP of clean vs new dataset...")
plot_umap(combined_embeddings, combined_labels, plot_3d=False)

# === Calculate distance of new images from clean cluster center ===
clean_center = np.mean(clean_embeddings, axis=0)
from sklearn.preprocessing import normalize

clean_embeddings_norm = normalize(clean_embeddings)
new_embeddings_norm = normalize(new_embeddings)
clean_center = np.mean(clean_embeddings_norm, axis=0).reshape(1, -1)
distances = pairwise_distances(new_embeddings_norm, clean_center)[:, 0]

distances = pairwise_distances(new_embeddings, [clean_center])[:,0]

# === Find top noisy candidates ===
#top_k = 10
#sorted_idx = np.argsort(distances)[::-1]  # descending order
#print(f"\nTop {top_k} most distant (potentially noisy) images from new dataset:")
#for i in sorted_idx[:top_k]:
 #   print(f"Distance: {distances[i]:.4f} - Path: {new_paths[i]}")

# === Optional: copy top noisy images to a folder for review ===
#COPY_NOISY = True
#NOISY_DIR = "./noisy_candidates"
#if COPY_NOISY:

#     os.makedirs(NOISY_DIR, exist_ok=True)
#     for i in sorted_idx[:top_k]:
#         src = new_paths[i]
#         dst = os.path.join(NOISY_DIR, os.path.basename(src))
#         shutil.copy2(src, dst)
#     print(f"\nCopied top {top_k} noisy images to folder: {NOISY_DIR}")

# # print("Done!")'''


In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 4))
sns.histplot(distances, bins=30, kde=True)
plt.axvline(np.mean(distances), color='green', linestyle='--', label='Mean')
plt.axvline(np.percentile(distances, 95), color='red', linestyle='--', label='95th percentile')
plt.xlabel("Distance to Clean Cluster Center")
plt.ylabel("Number of Images")
plt.title("Distribution of New Image Distances")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Sort indices by descending distance
sorted_indices = np.argsort(distances)[::-1]

print("Images sorted by distance (descending):")
for idx in sorted_indices:
    print(f"{new_paths[idx]} = {distances[idx]:.4f}")

with open("distances.txt", "w") as f:
    for idx in sorted_indices:
        f.write(f"{distances[idx]:.4f},{new_paths[idx]}\n")
